In [1]:
import os
import cv2
import imgaug as ia
import imgaug.augmenters as iaa
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing

In [3]:
# Directory with the original JPEG images
jpeg_dir = "pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/JPEGImages"
# Directory with the original Pascal_voc xml files
annotation_dir = "pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_NoAugmentation/Annotations"

# Directory to save the augmented JPEG images
aug_jpeg_dir = "pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_With-3X-Augmentation_no-rotation/JPEGImages"
# Directory to save the augmented Pascal_voc xml files
aug_annotation_dir = "pascal_voc_datasets/VOCdevkit/PlotsEnchanced_Original_With-3X-Augmentation_no-rotation/Annotations"

# Create directories if they don't exist
os.makedirs(aug_jpeg_dir, exist_ok=True)
os.makedirs(aug_annotation_dir, exist_ok=True)

# Define the augmentation sequence
seq = iaa.Sequential([
    iaa.Sometimes(0.5, iaa.Fliplr(0.5)),
    iaa.Sometimes(0.5, iaa.Flipud(0.5)),
    iaa.Sometimes(0.5, iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8, 1.2)})),
    iaa.Sometimes(0.5, iaa.AdditiveGaussianNoise(scale=0.1*255))  # add gaussian noise
])

def modify_xml(xml_path, image_aug, bbs_aug, new_xml_path, new_jpeg_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    root.find('folder').text = os.path.dirname(new_jpeg_path)
    root.find('filename').text = os.path.basename(new_jpeg_path)
    root.find('path').text = new_jpeg_path
    size = root.find('size')
    size.find('width').text = str(image_aug.shape[1])
    size.find('height').text = str(image_aug.shape[0])

    for i, obj in enumerate(root.iter('object')):
        bb = bbs_aug[i]
        xml_box = obj.find('bndbox')
        xml_box.find('xmin').text = str(bb.x1)
        xml_box.find('ymin').text = str(bb.y1)
        xml_box.find('xmax').text = str(bb.x2)
        xml_box.find('ymax').text = str(bb.y2)

    xml_str = ET.tostring(root, encoding='utf8')
    reparsed = minidom.parseString(xml_str)
    pretty_str = reparsed.toprettyxml(indent="\t")

    with open(new_xml_path, "w") as f:
        f.write(pretty_str)

# Number of augmented versions to generate per image
num_aug_versions = 3

# Parallel processing function
def process_image(filename):
    if filename.endswith(".jpg") or filename.endswith(".jpeg"):
        basename = os.path.splitext(filename)[0]
        jpeg_path = os.path.join(jpeg_dir, filename)
        xml_path = os.path.join(annotation_dir, basename + '.xml')

        # Load image
        image = cv2.imread(jpeg_path)
        # Convert BGR image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Load bounding boxes from XML file
        bbs = []
        tree = ET.parse(xml_path)
        root = tree.getroot()
        for member in root.findall('object'):
            bbox = member.find('bndbox')
            bbs.append(ia.BoundingBox(
                x1=int(bbox.find('xmin').text),
                y1=int(bbox.find('ymin').text),
                x2=int(bbox.find('xmax').text),
                y2=int(bbox.find('ymax').text)
            ))

        bbs = ia.BoundingBoxesOnImage(bbs, shape=image.shape)

        # Generate multiple augmented versions
        for i in range(num_aug_versions):
            # Apply augmentation
            image_aug, bbs_aug = seq(image=image, bounding_boxes=bbs)

            # Save the new JPEG file
            new_jpeg_path = os.path.join(aug_jpeg_dir, basename + f"_aug_{i}.jpg")
            # Convert RGB image back to BGR for saving
            image_aug_bgr = cv2.cvtColor(image_aug, cv2.COLOR_RGB2BGR)
            cv2.imwrite(new_jpeg_path, image_aug_bgr, [int(cv2.IMWRITE_JPEG_QUALITY), 60])

            # Save the new XML file
            new_xml_path = os.path.join(aug_annotation_dir, basename + f"_aug_{i}.xml")
            modify_xml(xml_path, image_aug, bbs_aug.bounding_boxes, new_xml_path, new_jpeg_path)


# Get list of files to process
files_to_process = os.listdir(jpeg_dir)

# Calculate the number of cores to use
num_cores = multiprocessing.cpu_count() // 2

# Process files in parallel
Parallel(n_jobs=num_cores)(delayed(process_image)(filename) for filename in tqdm(files_to_process))

100%|██████████████████████████████████████| 144888/144888 [18:40<00:00, 129.33it/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,